In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy

In [2]:
class MultiHeadAttention(nn.Module):
    #Initialize linear transformations of multihead attention q (query), k (key), v (value), and o (output)
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
    
    #Compute the scaled dot-product
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output

    #Reshape the input tensor into a 4-D tensor for the multiple attention heads.    
    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)

    #Combines the results of the attention heads, reshaping back to original shape.    
    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)

    #Forward pass    
    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output


In [3]:
#Processes and transforms position information in a sequence.
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        
        #Linear transformation that maps the input (d_model) to dimenstions of d_ff
        self.fc1 = nn.Linear(d_model, d_ff)

        #Linear transformation that maps d_ff to dimensions of d_model
        self.fc2 = nn.Linear(d_ff, d_model)

        #ReLU activation function application
        self.relu = nn.ReLU()

    #Forward pass, input x is passed through each of the above layers fc1, relu, then fc2.
    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [4]:
#Add positional encodings to the input sequences.
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        
        #Calculate positional encodings
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        
        #Compute sine and cosine of the positional encoding based on positional indices.
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        #Registers the 'pe' tensor as a buffer of this module so that the positional encodings are saved as part of the model's state.
        self.register_buffer('pe', pe.unsqueeze(0))

    #Forward pass where the positional encodings stored in self.pe are added to the input tensor 'x' with the sequence dimension.    
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]


In [5]:
#Processes input sequences and extracts relevant features
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        
        #From MultiHeadAttention, takes the input tensor as queries, keys, and values
        self.self_attn = MultiHeadAttention(d_model, num_heads)

        #From PositionWiseFeedForward, it's the position-wise feedforward neural network
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)

        #Layer normalization
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)

        #Dropout layer to prevent overfitting
        self.dropout = nn.Dropout(dropout)

    #Forward pass
    def forward(self, x, mask):

        #Input 'x' passed through multihead selfattention, results in an attention-weighted output
        attn_output = self.self_attn(x, x, x, mask)

        #Residual connection and layer normalization applied to above output
        x = self.norm1(x + self.dropout(attn_output))

        #Next is passed through the position-wise feedforward neural network
        ff_output = self.feed_forward(x)

        #Residual connection and layer normalization applied to output
        x = self.norm2(x + self.dropout(ff_output))
        return x


In [6]:
#Generates target sequences based on the input and previous target sequences.
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        #MultiHeadAttention module
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)

        #PositionWiseFeedForward module
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)

        #Layer normalization
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)

        #Dropout
        self.dropout = nn.Dropout(dropout)

    #Forward pass    
    def forward(self, x, enc_output, src_mask, tgt_mask):
        
        #Input 'x' passed through multihead selfattention, results in an attention-weighted output
        attn_output = self.self_attn(x, x, x, tgt_mask)

        #Residual connection and layer normalization applied to above output
        x = self.norm1(x + self.dropout(attn_output))

        #Cross-Attention, allowing decoder to attend to relevant information from encoder output
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)

        #Residual connection and layer normalization applied to above output
        x = self.norm2(x + self.dropout(attn_output))

        #Position-wise feed forward
        ff_output = self.feed_forward(x)

        #Residual connection and layer normalization applied to above output. Dropout applied after each layer.
        x = self.norm3(x + self.dropout(ff_output))
        return x


In [7]:
#Transformer model
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()

        #Embedding layer for the source language
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)

        #Embedding layer for the target language
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)

        #Positional encoding, adds positional information to the embeddings.
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        #Lists of encoders and decoder layers; each encoder/decoder layer calls  EncoderLayer and DecoderLayer respectively.
        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        #Linear transformation that maps the decoder output to the target vocab size; dropout applied to the output of hte positional encoding.
        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    #Creates src_mask and tgt_mask
    def generate_mask(self, src, tgt):
        #Used to mask positions in the source sequence that corresponds to the padding tokens.
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)

        #Used to mask positions int he target sequence corresponding to padding tokens and prevents them attending to future positions in the sequence.
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    #Forward pass
    def forward(self, src, tgt):

        #src_mask and tgt_mask are generated
        src_mask, tgt_mask = self.generate_mask(src, tgt)

        #source and target embeddings are passed through dropout and positional encoding.
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        #Encoder layers are applied to the source embeddings
        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        #Decoder layers are applied to the target embeddings using enc_output
        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        #dec_output passed through linear layer (self.fc)
        output = self.fc(dec_output)
        return output


In [8]:
src_vocab_size = 5000
tgt_vocab_size = 5000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 100
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

# Generate random sample data
src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)


In [9]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(100):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data[:, :-1])
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")


Epoch: 1, Loss: 8.683611869812012
Epoch: 2, Loss: 8.557395935058594
Epoch: 3, Loss: 8.478328704833984
Epoch: 4, Loss: 8.419227600097656
Epoch: 5, Loss: 8.359012603759766
Epoch: 6, Loss: 8.280267715454102
Epoch: 7, Loss: 8.19674015045166
Epoch: 8, Loss: 8.122048377990723
Epoch: 9, Loss: 8.031583786010742
Epoch: 10, Loss: 7.960486888885498
Epoch: 11, Loss: 7.866768836975098
Epoch: 12, Loss: 7.7862935066223145
Epoch: 13, Loss: 7.7043843269348145
Epoch: 14, Loss: 7.617316246032715
Epoch: 15, Loss: 7.538102626800537
Epoch: 16, Loss: 7.459163188934326
Epoch: 17, Loss: 7.3733954429626465
Epoch: 18, Loss: 7.2916035652160645
Epoch: 19, Loss: 7.204293251037598
Epoch: 20, Loss: 7.132760524749756
Epoch: 21, Loss: 7.040804386138916
Epoch: 22, Loss: 6.96254825592041
Epoch: 23, Loss: 6.888316631317139
Epoch: 24, Loss: 6.821549415588379
Epoch: 25, Loss: 6.756119728088379
Epoch: 26, Loss: 6.669528484344482
Epoch: 27, Loss: 6.595972061157227
Epoch: 28, Loss: 6.5228753089904785
Epoch: 29, Loss: 6.4507589